In [ ]:
from torchvision import models
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import os
from PIL import Image
from torchvision import models
from skimage import io
import numpy as np

In [ ]:
print(len(sorted(os.listdir("/content/drive/MyDrive/Human_AI/Output3"))))

160


In [ ]:
def file_len(fname):
    L = []
    with open(fname) as f:
        for i, l in enumerate(f):
            L.append(l.split()[1])
    return L

In [ ]:
file_len("/content/drive/MyDrive/Human_AI/val_2.txt")[120]

'516'

In [ ]:
directory = "/content/drive/MyDrive/Human_AI/Output2"

In [ ]:
class ImageData(Dataset):
    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image = []
        self.label = []
        for image in sorted(os.listdir(directory)):
          self.image.append(directory+'/'+image)
        f = open("/content/drive/MyDrive/Human_AI/val_2.txt", "r")
        for x in f:
          self.label.append(x.split()[1])
        self.transform = transform
    def __len__(self):  
      return len(self.image)
    def __getitem__(self, idx):
      image = self.image[idx]
      label = self.label[idx]
      if self.transform:
        img = self.transform(Image.open(image))
      else:
        img = io.imread(image)
      target = torch.tensor(int(label))
      return img, target


In [ ]:
vggnet = models.vgg19(pretrained=True)
vggnet.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
test = ImageData(transform=transforms.Compose([
                                               transforms.Resize([224, 224]),
                                              transforms.ToTensor()]))
test_data = DataLoader(test, batch_size=64,shuffle=False, num_workers=1,pin_memory=True)

In [ ]:
vggnet.eval()
score = []
for i, data in enumerate(test_data):
    images, labels = data
    tmp = []
    tmp = torch.squeeze(labels.long())
    images, labels = images.cuda(),  tmp.cuda()
    outputs = vggnet(images)
    outputs_numpy = outputs.cpu().data.numpy()
    outputs_argmax = np.argmax(outputs_numpy,axis=1)
    #print(outputs_argmax)
    labels_numpy = labels.cpu().data.numpy()
    #print(labels_numpy)
    score = np.concatenate((score,(labels_numpy==outputs_argmax).astype(int)),axis=0)
    #break

meanAccuracy3 = sum(score)/len(score)


In [ ]:
print(meanAccuracy3)

0.05625


In [ ]:
len(score)

64

In [ ]:
print(meanAccuracy2)

0.4375


In [ ]:
print(outputs_argmax)

[ 77 610 459 636 662 213 983 427 212 537  35 741   6 847 289 425 973 115
 167 121 876 602 532 295 678 840  70 348  13 180 794 894]


In [ ]:
print(labels_numpy)

[ 77 610 254 636 662 213  25 463 215 173  35 741 125 787 289 425 973   1
 167 121 445 702 532 366 678 764 125 349  13 179 522 493]


In [ ]:
print(meanAccuracy)

0.01875
